# Building and Populating the NHANES SQLite Database for Integrated Data Analysis

## Step 1: Import and Set Up

***Setting Up Environment for Database Operations***

Initializes the Python environment by importing essential libraries and custom project modules. It adjusts the system path to include the project root, allowing access to modules within the 'scripts' directory. This setup 
enables seamless interaction with the SQLite database, management of NHANES data tables, and streamlined handling of file paths.

In [1]:
# 1. Setup environment and imports
import pandas as pd
import numpy as np
from pathlib import Path

import sys

# --- Add project root (parent of 'scripts') to sys.path
current_dir = Path().resolve()
project_root = current_dir.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from scripts.db_utils import create_connection_from_script, save_to_sqlite, close_connection, run_query, create_nhanes_tables
from scripts.config import (
    PROCESSED_DATA_DIR,
    DATABASE_PATH, 
    FINAL_DATA_DIR    
)
from scripts.utils import pretty_path, explore_data

print("Setup complete.")

Setup complete.


## Step 2: Define Your Required Files and Paths

***Define Processed NHANES Datasets for Database Loading***

I created a dictionary mapping human-readable dataset names to their corresponding processed CSV filenames. These files are the cleaned and feature-engineered NHANES data and will be used for loading into the SQLite database. This setup makes it easier to loop through and handle each dataset consistently.

In [2]:

# List of processed CSV files
datasets_to_load = {
    "demographics": "demo_l_processed.csv",
    "diet": "hei2015_scores.csv",
    "physical_activity": "paq_l_processed.csv",
    "sleep": "slq_l_processed.csv",
    "health_insurance": "hiq_l_processed.csv",
    "bmi": "bmx_l_processed.csv",
    "bp": "bpxo_l_processed.csv",
    "total_cholestrol": "tchol_l_processed.csv",
    "glucose": "glu_l_processed.csv",
    "diabetes": "diq_l_processed.csv",
    "cardio_vascular": "mcq_l_processed.csv"
}


## Step 3: Load Each CSV into a DataFrame

***Load and Clean Processed CSV Files into DataFrames***

I loaded each processed NHANES CSV into a Pandas DataFrame and stored them in a dictionary called dataframes. While loading, I made sure that participant_id is treated as a string and cleaned it by removing any unnecessary decimal points (like converting "12345.0" to "12345"). This ensures consistency before pushing the data to the database.

In [3]:
# Dictionary to store DataFrames
dataframes = {}

for table_name, filename in datasets_to_load.items():
    file_path = Path(PROCESSED_DATA_DIR) / filename

    # Load CSV, force participant_id column to string
    df = pd.read_csv(file_path, dtype={"participant_id": str})

    # Check if any participant_id values look like float strings (e.g., "12345.0")
    if df['participant_id'].str.contains(r'\.0+$').any():
        print(f"Cleaning 'participant_id' in '{table_name}'...")
        df['participant_id'] = df['participant_id'].apply(
            lambda x: str(int(float(x))) if pd.notnull(x) else np.nan
        )

    dataframes[table_name] = df
    print(f"Loaded '{table_name}' with shape {df.shape}")


Loaded 'demographics' with shape (6064, 12)
Loaded 'diet' with shape (6739, 18)
Loaded 'physical_activity' with shape (5894, 9)
Loaded 'sleep' with shape (8371, 5)
Loaded 'health_insurance' with shape (11871, 2)
Loaded 'bmi' with shape (8471, 3)
Loaded 'bp' with shape (7518, 4)
Loaded 'total_cholestrol' with shape (6890, 4)
Loaded 'glucose' with shape (3360, 8)
Loaded 'diabetes' with shape (11744, 5)
Loaded 'cardio_vascular' with shape (11744, 6)


###  Step 4: Connect to SQLite Database

***Create SQLite Database Connection***

I used the create_connection_from_script() function to establish a connection to the SQLite database located at DATABASE_PATH. This connection (conn) will be used to create tables and insert the processed data into the database.

In [4]:
conn = create_connection_from_script(DATABASE_PATH)

Connected to database at: database\nhanes_2021_2023.db


###  Step 5: Create Tables

***Create NHANES Tables in SQLite Database***

I called create_nhanes_tables(conn) to create the necessary empty tables in the database using the active connection. This sets up the structure needed to insert the processed NHANES data (like demographics, diet, activity, etc.) into the database.

In [5]:
create_nhanes_tables(conn)

Created table 'demographics'
Created table 'health_insurance'
Created table 'sleep'
Created table 'physical_activity'
Created table 'diet'
Created table 'bmi'
Created table 'bp'
Created table 'total_cholestrol'
Created table 'glucose'
Created table 'diabetes'
Created table 'cardio_vascular'
All required NHANES tables created successfully.


### Check point 1: Check tables in the database

***Check Available Tables in Database***

I ran a query to list all the tables currently created in the SQLite database. This confirms the tables were successfully set up before inserting data. The printed output will show their names so I can double-check everything is ready to go.

In [6]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = run_query(conn, query)

print("Tables in the database:")
print(tables_df)

Tables in the database:
                 name
0        demographics
1               sleep
2                diet
3   physical_activity
4    health_insurance
5                 bmi
6                  bp
7    total_cholestrol
8             glucose
9            diabetes
10    cardio_vascular


### Check point 2: Get the structure of each table

***Review Table Schemas in SQLite Database***

I pulled the list of all tables from the NHANES database, then looped through each one to print its schema. This helps me understand the structure—what columns each table has, their data types, and whether any are primary keys. It's basically a sanity check before loading any data in.

In [7]:
# Get the list of tables
tables_df = run_query(conn, "SELECT name FROM sqlite_master WHERE type='table';")
table_names = tables_df['name'].tolist()

# Loop through each table and print its schema
for table in table_names:
    print(f"\nSchema for table: '{table}'")
    schema_df = run_query(conn, f"PRAGMA table_info({table});")
    print(schema_df)


Schema for table: 'demographics'
    cid                     name     type  notnull dflt_value  pk
0     0           participant_id     TEXT        0       None   1
1     1                   gender     TEXT        0       None   0
2     2                      age  INTEGER        0       None   0
3     3           race_ethnicity     TEXT        0       None   0
4     4          education_level     TEXT        0       None   0
5     5     poverty_income_ratio     REAL        0       None   0
6     6  interview_sample_weight     REAL        0       None   0
7     7       exam_sample_weight     REAL        0       None   0
8     8                   strata     REAL        0       None   0
9     9                      psu     REAL        0       None   0
10   10             pir_category     TEXT        0       None   0

Schema for table: 'sleep'
   cid            name  type  notnull dflt_value  pk
0    0  participant_id  TEXT        0       None   1
1    1    sleep_avg_hr  REAL        0    

   cid            name  type  notnull dflt_value  pk
0    0  participant_id  TEXT        0       None   1
1    1    systolic_avg  REAL        0       None   0
2    2   diastolic_avg  REAL        0       None   0
3    3     bp_category  TEXT        0       None   0

Schema for table: 'total_cholestrol'
   cid                       name  type  notnull dflt_value  pk
0    0             participant_id  TEXT        0       None   1
1    1          total_cholesterol  REAL        0       None   0
2    2  blood_drawn_sample_weight  REAL        0       None   0
3    3       cholesterol_category  TEXT        0       None   0

Schema for table: 'glucose'
   cid                       name     type  notnull dflt_value  pk
0    0             participant_id     TEXT        0       None   1
1    1      fasting_glucose_mg_dl     REAL        0       None   0
2    2   fasting_subsample_weight     REAL        0       None   0
3    3           glucose_category     TEXT        0       None   0
4    4       

### Step 6: Save DataFrames to SQLite Tables

***Saving Cleaned NHANES Datasets to SQLite***

All the cleaned NHANES datasets were saved into a SQLite database. Each dataset (like demographics, sleep, diet, etc.) was saved as a separate table. If a table already existed, it was replaced with the new one. This step helps store the data in one place so it can be easily used later for analysis.

In [8]:
# Save data
save_to_sqlite(df=dataframes["demographics"], conn=conn, table_name="demographics", recreate=True)
save_to_sqlite(df=dataframes["sleep"], conn=conn, table_name="sleep", recreate=True)
save_to_sqlite(df=dataframes["diet"], conn=conn, table_name="diet", recreate=True)
save_to_sqlite(df=dataframes["physical_activity"], conn=conn, table_name="physical_activity", recreate=True)
save_to_sqlite(df=dataframes["health_insurance"], conn=conn, table_name="health_insurance", recreate=True)

save_to_sqlite(df=dataframes["bmi"], conn=conn, table_name="bmi", recreate=True)
save_to_sqlite(df=dataframes["bp"], conn=conn, table_name="bp", recreate=True)
save_to_sqlite(df=dataframes["total_cholestrol"], conn=conn, table_name="total_cholestrol", recreate=True)
save_to_sqlite(df=dataframes["glucose"], conn=conn, table_name="glucose", recreate=True)
save_to_sqlite(df=dataframes["diabetes"], conn=conn, table_name="diabetes", recreate=True)
save_to_sqlite(df=dataframes["cardio_vascular"], conn=conn, table_name="cardio_vascular", recreate=True)

Recreated table 'demographics'
Dropping columns not in table 'demographics': ['interview_sample_weight_missing']
Inserted 6064 rows into 'demographics'
Recreated table 'sleep'
Dropping columns not in table 'sleep': ['sleep_weekday_hr', 'sleep_weekend_hr']
Inserted 8371 rows into 'sleep'
Recreated table 'diet'
Dropping columns not in table 'diet': ['hei_added_sugars', 'hei_dairy', 'hei_fatty_acid', 'hei_greens_beans', 'hei_refined_grains', 'hei_sat_fats', 'hei_sea_plant_protein', 'hei_sodium', 'hei_total_fruit', 'hei_total_protein', 'hei_total_veg', 'hei_whole_fruit', 'hei_whole_grains']
Inserted 6739 rows into 'diet'
Recreated table 'physical_activity'
Dropping columns not in table 'physical_activity': ['duration_min', 'freq', 'freq_per_week', 'freq_unit', 'sedentary_min_per_day']
Inserted 5894 rows into 'physical_activity'
Recreated table 'health_insurance'
Inserted 11871 rows into 'health_insurance'
Recreated table 'bmi'
Inserted 8471 rows into 'bmi'
Recreated table 'bp'
Inserted 751

### Step 7: Check point

***Preview of Demographics Table from Database***

A sample of the first 5 rows from the demographics table was retrieved from the SQLite database using a simple SQL query. This helps confirm that the data was saved correctly and gives a quick look at the structure and values in the table.

In [9]:
demo = run_query(conn, "SELECT * FROM demographics LIMIT 5;")
print(demo)


  participant_id  gender  age      race_ethnicity            education_level  \
0         130378    Male   43  Non-Hispanic Asian  College graduate or above   
1         130379    Male   66  Non-Hispanic White  College graduate or above   
2         130380  Female   44      Other Hispanic            High school/GED   
3         130386    Male   34    Mexican American     Some college/AA degree   
4         130387  Female   68  Non-Hispanic White  College graduate or above   

   poverty_income_ratio  interview_sample_weight  exam_sample_weight  strata  \
0                  5.00             50055.450807        54374.463898   173.0   
1                  5.00             29087.450605        34084.721548   173.0   
2                  1.41             80062.674301        81196.277992   174.0   
3                  1.33             30995.282610        39988.452940   179.0   
4                  1.32             19896.970559        20776.254850   181.0   

   psu pir_category  
0  2.0    Very H

### Step 8: prepare the data for Objective 1.1 analysis using Sql query

***Merged Dataset of Lifestyle and Socio-Economic Indicators***

A SQL query joins the demographics table with lifestyle-related data from health_insurance, sleep, physical_activity, and diet tables using participant_id as the common key. The resulting dataset (ls_sei_df) includes key socio-economic and lifestyle variables such as age, gender, education, insurance status, sleep patterns, physical activity, and diet quality. This merged view is useful for analyzing how lifestyle and socio-economic factors are interrelated.

In [10]:
join_query = """
SELECT
    d.participant_id,
    d.age,
    d.gender,
    d.race_ethnicity,
    d.education_level,
    d.poverty_income_ratio,
    d.pir_category,
    d.interview_sample_weight,  
    d.psu,
    d.strata,  
    i.has_health_insurance,
    s.sleep_avg_hr,
    s.sleep_category,
    p.activity_level,
    p.total_weekly_min,
    di.hei_score,
    di.diet_score_category,
    di.total_diet_weight,
    di.food_item_weight
FROM demographics d
LEFT JOIN health_insurance i ON d.participant_id = i.participant_id
LEFT JOIN sleep s ON d.participant_id = s.participant_id
LEFT JOIN physical_activity p ON d.participant_id = p.participant_id
LEFT JOIN diet di ON d.participant_id = di.participant_id
"""

ls_sei_df = run_query(conn, join_query) # ls_sei - lifestyle and socio-economic indicator
ls_sei_df.head()

,participant_id,age,gender,race_ethnicity,education_level,poverty_income_ratio,pir_category,interview_sample_weight,psu,strata,has_health_insurance,sleep_avg_hr,sleep_category,activity_level,total_weekly_min,hei_score,diet_score_category,total_diet_weight,food_item_weight
0,130378,43,Male,Non-Hispanic Asian,College graduate or above,5.00,Very High,50055.450807,2.0,173.0,Yes,9.357143,Long Sleep,Low active,135.0,41.572532,Poor,61366.555827,6.750321e+05
1,130379,66,Male,Non-Hispanic White,College graduate or above,5.00,Very High,29087.450605,2.0,173.0,Yes,9.000000,Normal Sleep,Moderately active,180.0,70.282500,Needs Improvement,34638.056480,5.888470e+05
2,130380,44,Female,Other Hispanic,High school/GED,1.41,Mid,80062.674301,1.0,174.0,Yes,8.285714,Normal Sleep,Low active,20.0,56.099703,Poor,84728.261560,1.101467e+06
3,130386,34,Male,Mexican American,Some college/AA degree,1.33,Mid,30995.282610,1.0,179.0,Yes,7.642857,Normal Sleep,Low active,30.0,36.878689,Poor,82013.365563,7.381203e+05
4,130387,68,Female,Non-Hispanic White,College graduate or above,1.32,Mid,19896.970559,1.0,181.0,Yes,3.571429,Short Sleep,None,NaN,53.563152,Poor,20032.615949,4.206849e+05


### Step 9: Check and Save the lifestyle and socio-economic Merged Data

***Saving Final Merged Lifestyle and Socioeconomic Dataset***

I explored the merged lifestyle and socioeconomic data, saved it as a CSV file in the final data folder, and confirmed the file was saved successfully.

In [11]:
# Explore and Understand the merged Data
print("Exploring Merged Final Merged Lifestyle and Socioeconomic Dataset..")
explore_data(ls_sei_df)

Exploring Merged Final Merged Lifestyle and Socioeconomic Dataset..

--- Exploring Dataset ---
Shape: (6064, 19)

First 5 rows:
  participant_id  age  gender      race_ethnicity            education_level  \
0         130378   43    Male  Non-Hispanic Asian  College graduate or above   
1         130379   66    Male  Non-Hispanic White  College graduate or above   
2         130380   44  Female      Other Hispanic            High school/GED   
3         130386   34    Male    Mexican American     Some college/AA degree   
4         130387   68  Female  Non-Hispanic White  College graduate or above   

   poverty_income_ratio pir_category  interview_sample_weight  psu  strata  \
0                  5.00    Very High             50055.450807  2.0   173.0   
1                  5.00    Very High             29087.450605  2.0   173.0   
2                  1.41          Mid             80062.674301  1.0   174.0   
3                  1.33          Mid             30995.282610  1.0   179.0   
4

In [12]:
# Define the output path
output_path = FINAL_DATA_DIR / "final_merged_lifestyle_socio_economic.csv"
ls_sei_df.to_csv(output_path, index=False)

# Check if the file exists using pathlib
if output_path.exists():
    print(f"File saved successfully at: {pretty_path(output_path)}")
else:
    print(f"File was not saved at: {pretty_path(output_path)}")

File saved successfully at: data\final\final_merged_lifestyle_socio_economic.csv


### Step 9: Prepare the full merged dataset for further analysing the objectives

***Merging Additional NHANES Tables into Main Dataset***

I merged six more NHANES tables (bmi, bp, cholesterol, glucose, diabetes, cardiovascular) with the existing lifestyle and socioeconomic data using participant IDs. Then, I added exam sample weights from demographics to complete the combined dataset for further analysis.

In [13]:
remaining_tables = [
    "bmi",
    "bp",
    "total_cholestrol",
    "glucose",
    "diabetes",
    "cardio_vascular"
]
nhanes_df = ls_sei_df.copy()

for table in remaining_tables:
    df = run_query(conn, f"SELECT * FROM {table}")
    
    if 'participant_id' not in df.columns:
        raise KeyError(f"'participant_id' not found in table '{table}'")
    
    nhanes_df = pd.merge(nhanes_df, df, on="participant_id", how="left")

# Merge exam_sample_weight from demographics table
query = "SELECT participant_id, exam_sample_weight FROM demographics"
demo_df = run_query(conn, query)

# Safety check
if 'participant_id' not in demo_df.columns:
    raise KeyError("'participant_id' not found in 'demographics' table")

# Merge with your main NHANES dataframe
nhanes_df = pd.merge(nhanes_df, demo_df, on="participant_id", how="left")

nhanes_df.head()


,participant_id,age,gender,race_ethnicity,education_level,poverty_income_ratio,pir_category,interview_sample_weight,psu,strata,...,diabetes_dx,diabetes_meds,diabetes_meds_cat,diabetes_status,congestive_heart_failure,coronary_heart_disease,angina,heart_attack,any_cvd,exam_sample_weight
0,130378,43,Male,Non-Hispanic Asian,College graduate or above,5.00,Very High,50055.450807,2.0,173.0,...,0.0,NaN,Unknown,0.0,0.0,0.0,0.0,0.0,0,54374.463898
1,130379,66,Male,Non-Hispanic White,College graduate or above,5.00,Very High,29087.450605,2.0,173.0,...,0.0,NaN,Unknown,0.0,0.0,0.0,0.0,0.0,0,34084.721548
2,130380,44,Female,Other Hispanic,High school/GED,1.41,Mid,80062.674301,1.0,174.0,...,1.0,1.0,Taking meds,1.0,0.0,0.0,0.0,0.0,0,81196.277992
3,130386,34,Male,Mexican American,Some college/AA degree,1.33,Mid,30995.282610,1.0,179.0,...,0.0,NaN,Unknown,0.0,0.0,0.0,0.0,0.0,0,39988.452940
4,130387,68,Female,Non-Hispanic White,College graduate or above,1.32,Mid,19896.970559,1.0,181.0,...,0.0,NaN,Unknown,0.0,0.0,0.0,0.0,0.0,0,20776.254850


### Step 10: Check and Save the full NHANES Data

***Saving the Complete Merged NHANES Dataset***

I explored the fully merged NHANES dataset and  saved it as a CSV file in the final data folder, and confirmed the file was saved successfully.

In [14]:
# Explore and Understand the final merged Data
print("Exploring final Merged NHANES data..")
explore_data(nhanes_df)

Exploring final Merged NHANES data..

--- Exploring Dataset ---
Shape: (6064, 43)

First 5 rows:
  participant_id  age  gender      race_ethnicity            education_level  \
0         130378   43    Male  Non-Hispanic Asian  College graduate or above   
1         130379   66    Male  Non-Hispanic White  College graduate or above   
2         130380   44  Female      Other Hispanic            High school/GED   
3         130386   34    Male    Mexican American     Some college/AA degree   
4         130387   68  Female  Non-Hispanic White  College graduate or above   

   poverty_income_ratio pir_category  interview_sample_weight  psu  strata  \
0                  5.00    Very High             50055.450807  2.0   173.0   
1                  5.00    Very High             29087.450605  2.0   173.0   
2                  1.41          Mid             80062.674301  1.0   174.0   
3                  1.33          Mid             30995.282610  1.0   179.0   
4                  1.32         

In [15]:
# Define the output path
output_path = FINAL_DATA_DIR / "final_merged_nhanes_dataset.csv"
nhanes_df.to_csv(output_path, index=False)
# Check if the file exists 
if output_path.exists():
    print(f"File saved successfully at: {pretty_path(output_path)}")
else:
    print(f"File was not saved at: {pretty_path(output_path)}")

File saved successfully at: data\final\final_merged_nhanes_dataset.csv


### Step 11: Close the database connection

***Closing the Database Connection After Data Processing***

I closed the connection to the SQLite database to properly release resources after completing all data loading, merging, and saving tasks. This step ensures the database is cleanly closed and ready for future use without any locked or lingering connections.

In [16]:
close_connection(conn)

Database connection closed.
